In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# Add checkpointer for persistence
checkpointer = MemorySaver()

@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    return f"Weather in {city}: 72°F, sunny"
gptLLM = ChatOpenAI(model_name="gpt-4",temperature=0)
ollamaLLM = ChatOllama(model="llama3-groq-tool-use")

agent = create_agent(
    model=ollamaLLM,
    tools=[get_weather],
    system_prompt="You are a helpful assistant.",
    checkpointer=checkpointer  # Saves state automatically
)

# Thread ID persists conversation across sessions
config = {"configurable": {"thread_id": "user_123"}}

# First invocation
result1 = agent.invoke(
    {"messages": [HumanMessage(content="My Name is Ron. What's the weather like in New York City?")]},
    config
)
print(result1["messages"][-1].content)

# Later session - remembers everything!
result2 = agent.invoke(
    {"messages": [HumanMessage(content="What is my Name?")]},
    config  # Same thread_id = same memory
)
print(result2["messages"][-1].content)